# Bongkaran to Pengajuan Empty 20 DC

In [16]:
# Step 1: Import Semua Library yang Dibutuhkan
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

# Library untuk Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder


# Step 2: Membaca dan Membersihkan Data Awal
# Membaca file
df = pd.read_excel('Ship_Operation_Data_Cleaned_1.xlsx')

# Tentukan fitur dan target
kolom_fitur_kategorikal = ['VESSEL ID (DMY)', 'BERTH LOCATION']
kolom_fitur_numerik = ['Voyage No.', 'Voyage Yr', 'TOTAL BONGKARAN_EMPTY_20 DC']
kolom_fitur = kolom_fitur_kategorikal + kolom_fitur_numerik
kolom_target = 'PENGAJUAN KE PLANNER_EMPTY_20 DC'

print(f"\nTarget yang akan diprediksi: '{kolom_target}'")

print("\nMembersihkan kolom fitur numerik...")
for col in kolom_fitur_numerik:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    print(f"  - Kolom '{col}' telah dipastikan menjadi numerik.")

# Membersihkan data di kolom target (fitur akan ditangani oleh Pipeline)
df[kolom_target] = pd.to_numeric(df[kolom_target], errors='coerce')
if df[kolom_target].isnull().sum() > 0:
    median_value = df[kolom_target].median()
    df[kolom_target].fillna(median_value, inplace=True)
    print(f"  - Nilai kosong di '{kolom_target}' diisi dengan median ({median_value}).")


# Step 3: Membuat Pipeline Preprocessing dan Model
# Definisikan untuk kolom numerik: isi nilai kosong dengan median.
numeric_transformer = SimpleImputer(strategy='median')

# Definisikan untuk kolom kategorikal: isi nilai kosong lalu lakukan one-hot encoding.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')) # 'ignore' agar tidak error jika ada kategori baru saat prediksi
])

# Gabungkan kedua di atas menjadi satu preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, kolom_fitur_numerik),
        ('cat', categorical_transformer, kolom_fitur_kategorikal)
    ])

# Buat Pipeline Lengkap: rangkai preprocessor dengan model Random Forest
full_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', RandomForestRegressor(random_state=42))
])

print("\nPipeline untuk preprocessing dan model Random Forest berhasil dibuat.")


# Step 4: Membagi Data dan Melakukan GridSearchCV pada Pipeline
# Pisahkan fitur (X) dan target (y)
X = df[kolom_fitur]
y = df[kolom_target]

# Bagi data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nData dibagi menjadi:")
print(f"  - {X_train.shape[0]} baris untuk training")
print(f"  - {X_test.shape[0]} baris untuk testing")

# Tentukan parameter yang ingin diuji untuk model di dalam pipeline
# PENTING: nama parameter harus diawali dengan nama langkah di pipeline, yaitu 'model__'
param_grid_rf = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [10, 20, 30],
    'model__min_samples_split': [2, 5],
    'model__min_samples_leaf': [1, 2]
}

# Siapkan GridSearchCV untuk dijalankan pada SELURUH pipeline
grid_search = GridSearchCV(estimator=full_pipeline, param_grid=param_grid_rf, cv=3, n_jobs=-1, scoring='r2', verbose=2)

print("\nMemulai pencarian parameter terbaik untuk Random Forest di dalam pipeline...")
start_time = time.time()
grid_search.fit(X_train, y_train)
training_time = time.time() - start_time


# Step 5: Menampilkan Hasil Evaluasi
# Ambil pipeline terbaik yang ditemukan
best_pipeline = grid_search.best_estimator_

# Lakukan prediksi menggunakan pipeline terbaik
y_pred = best_pipeline.predict(X_test)

# Hitung semua metrik
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Tampilkan semua hasil
print("\nHASIL EVALUASI MODEL RANDOM FOREST")
print(f"Waktu training          : {training_time:.2f} detik")
print(f"Parameter terbaik        : {grid_search.best_params_}")
print("-" * 50)
print(f"R-squared (R²)           : {r2:.4f}")
print(f"Mean Absolute Error (MAE)  : {mae:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")


Target yang akan diprediksi: 'PENGAJUAN KE PLANNER_EMPTY_20 DC'

Membersihkan kolom fitur numerik...
  - Kolom 'Voyage No.' telah dipastikan menjadi numerik.
  - Kolom 'Voyage Yr' telah dipastikan menjadi numerik.
  - Kolom 'TOTAL BONGKARAN_EMPTY_20 DC' telah dipastikan menjadi numerik.

Pipeline untuk preprocessing dan model Random Forest berhasil dibuat.

Data dibagi menjadi:
  - 1076 baris untuk training
  - 270 baris untuk testing

Memulai pencarian parameter terbaik untuk Random Forest di dalam pipeline...
Fitting 3 folds for each of 24 candidates, totalling 72 fits

HASIL EVALUASI MODEL RANDOM FOREST
Waktu training          : 45.59 detik
Parameter terbaik        : {'model__max_depth': 10, 'model__min_samples_leaf': 2, 'model__min_samples_split': 5, 'model__n_estimators': 100}
--------------------------------------------------
R-squared (R²)           : 0.7762
Mean Absolute Error (MAE)  : 19.0807
Root Mean Squared Error (RMSE): 35.2126


In [17]:
import joblib

joblib.dump(best_pipeline, 'bongkaran_pengajuan_empty_20.pkl')

['bongkaran_pengajuan_empty_20.pkl']

# Bongkaran to Pengajuan Empty 40 HC

In [18]:
# Step 1: Import Semua Library yang Dibutuhkan
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

# Library untuk Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder


# Step 2: Membaca dan Membersihkan Data Awal
# Membaca file
df = pd.read_excel('Ship_Operation_Data_Cleaned_1.xlsx')

# Tentukan fitur dan target
kolom_fitur_kategorikal = ['VESSEL ID (DMY)', 'BERTH LOCATION']
kolom_fitur_numerik = ['Voyage No.', 'Voyage Yr', 'TOTAL BONGKARAN_EMPTY_40 HC']
kolom_fitur = kolom_fitur_kategorikal + kolom_fitur_numerik
kolom_target = 'PENGAJUAN KE PLANNER_EMPTY_40 HC'

print(f"\nTarget yang akan diprediksi: '{kolom_target}'")

print("\nMembersihkan kolom fitur numerik...")
for col in kolom_fitur_numerik:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    print(f"  - Kolom '{col}' telah dipastikan menjadi numerik.")

# Membersihkan data di kolom target (fitur akan ditangani oleh Pipeline)
df[kolom_target] = pd.to_numeric(df[kolom_target], errors='coerce')
if df[kolom_target].isnull().sum() > 0:
    median_value = df[kolom_target].median()
    df[kolom_target].fillna(median_value, inplace=True)
    print(f"  - Nilai kosong di '{kolom_target}' diisi dengan median ({median_value}).")


# Step 3: Membuat Pipeline Preprocessing dan Model
# Definisikan untuk kolom numerik: isi nilai kosong dengan median.
numeric_transformer = SimpleImputer(strategy='median')

# Definisikan untuk kolom kategorikal: isi nilai kosong lalu lakukan one-hot encoding.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')) # 'ignore' agar tidak error jika ada kategori baru saat prediksi
])

# Gabungkan kedua di atas menjadi satu preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, kolom_fitur_numerik),
        ('cat', categorical_transformer, kolom_fitur_kategorikal)
    ])

# Buat Pipeline Lengkap: rangkai preprocessor dengan model Random Forest
full_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', RandomForestRegressor(random_state=42))
])

print("\nPipeline untuk preprocessing dan model Random Forest berhasil dibuat.")


# Step 4: Membagi Data dan Melakukan GridSearchCV pada Pipeline
# Pisahkan fitur (X) dan target (y)
X = df[kolom_fitur]
y = df[kolom_target]

# Bagi data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nData dibagi menjadi:")
print(f"  - {X_train.shape[0]} baris untuk training")
print(f"  - {X_test.shape[0]} baris untuk testing")

# Tentukan parameter yang ingin diuji untuk model di dalam pipeline
# PENTING: nama parameter harus diawali dengan nama langkah di pipeline, yaitu 'model__'
param_grid_rf = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [10, 20, 30],
    'model__min_samples_split': [2, 5],
    'model__min_samples_leaf': [1, 2]
}

# Siapkan GridSearchCV untuk dijalankan pada SELURUH pipeline
grid_search = GridSearchCV(estimator=full_pipeline, param_grid=param_grid_rf, cv=3, n_jobs=-1, scoring='r2', verbose=2)

print("\nMemulai pencarian parameter terbaik untuk Random Forest di dalam pipeline...")
start_time = time.time()
grid_search.fit(X_train, y_train)
training_time = time.time() - start_time


# Step 5: Menampilkan Hasil Evaluasi
# Ambil pipeline terbaik yang ditemukan
best_pipeline = grid_search.best_estimator_

# Lakukan prediksi menggunakan pipeline terbaik
y_pred = best_pipeline.predict(X_test)

# Hitung semua metrik
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Tampilkan semua hasil
print("\nHASIL EVALUASI MODEL RANDOM FOREST")
print(f"Waktu training          : {training_time:.2f} detik")
print(f"Parameter terbaik        : {grid_search.best_params_}")
print("-" * 50)
print(f"R-squared (R²)           : {r2:.4f}")
print(f"Mean Absolute Error (MAE)  : {mae:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")


Target yang akan diprediksi: 'PENGAJUAN KE PLANNER_EMPTY_40 HC'

Membersihkan kolom fitur numerik...
  - Kolom 'Voyage No.' telah dipastikan menjadi numerik.
  - Kolom 'Voyage Yr' telah dipastikan menjadi numerik.
  - Kolom 'TOTAL BONGKARAN_EMPTY_40 HC' telah dipastikan menjadi numerik.

Pipeline untuk preprocessing dan model Random Forest berhasil dibuat.

Data dibagi menjadi:
  - 1076 baris untuk training
  - 270 baris untuk testing

Memulai pencarian parameter terbaik untuk Random Forest di dalam pipeline...
Fitting 3 folds for each of 24 candidates, totalling 72 fits

HASIL EVALUASI MODEL RANDOM FOREST
Waktu training          : 37.19 detik
Parameter terbaik        : {'model__max_depth': 10, 'model__min_samples_leaf': 2, 'model__min_samples_split': 2, 'model__n_estimators': 100}
--------------------------------------------------
R-squared (R²)           : 0.5916
Mean Absolute Error (MAE)  : 7.5979
Root Mean Squared Error (RMSE): 14.9655


In [19]:
import joblib

joblib.dump(best_pipeline, 'bongkaran_pengajuan_empty_40.pkl')

['bongkaran_pengajuan_empty_40.pkl']

# Bongkaran to Pengajuan Full 20 DC

In [20]:
# Step 1: Import Semua Library yang Dibutuhkan
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

# Library untuk Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder


# Step 2: Membaca dan Membersihkan Data Awal
# Membaca file
df = pd.read_excel('Ship_Operation_Data_Cleaned_1.xlsx')

# Tentukan fitur dan target
kolom_fitur_kategorikal = ['VESSEL ID (DMY)', 'BERTH LOCATION']
kolom_fitur_numerik = ['Voyage No.', 'Voyage Yr', 'TOTAL BONGKARAN_FULL_20 DC']
kolom_fitur = kolom_fitur_kategorikal + kolom_fitur_numerik
kolom_target = 'PENGAJUAN KE PLANNER_FULL_20 DC'

print(f"\nTarget yang akan diprediksi: '{kolom_target}'")

print("\nMembersihkan kolom fitur numerik...")
for col in kolom_fitur_numerik:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    print(f"  - Kolom '{col}' telah dipastikan menjadi numerik.")

# Membersihkan data di kolom target (fitur akan ditangani oleh Pipeline)
df[kolom_target] = pd.to_numeric(df[kolom_target], errors='coerce')
if df[kolom_target].isnull().sum() > 0:
    median_value = df[kolom_target].median()
    df[kolom_target].fillna(median_value, inplace=True)
    print(f"  - Nilai kosong di '{kolom_target}' diisi dengan median ({median_value}).")


# Step 3: Membuat Pipeline Preprocessing dan Model
# Definisikan untuk kolom numerik: isi nilai kosong dengan median.
numeric_transformer = SimpleImputer(strategy='median')

# Definisikan untuk kolom kategorikal: isi nilai kosong lalu lakukan one-hot encoding.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')) # 'ignore' agar tidak error jika ada kategori baru saat prediksi
])

# Gabungkan kedua di atas menjadi satu preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, kolom_fitur_numerik),
        ('cat', categorical_transformer, kolom_fitur_kategorikal)
    ])

# Buat Pipeline Lengkap: rangkai preprocessor dengan model Random Forest
full_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', RandomForestRegressor(random_state=42))
])

print("\nPipeline untuk preprocessing dan model Random Forest berhasil dibuat.")


# Step 4: Membagi Data dan Melakukan GridSearchCV pada Pipeline
# Pisahkan fitur (X) dan target (y)
X = df[kolom_fitur]
y = df[kolom_target]

# Bagi data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nData dibagi menjadi:")
print(f"  - {X_train.shape[0]} baris untuk training")
print(f"  - {X_test.shape[0]} baris untuk testing")

# Tentukan parameter yang ingin diuji untuk model di dalam pipeline
# PENTING: nama parameter harus diawali dengan nama langkah di pipeline, yaitu 'model__'
param_grid_rf = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [10, 20, 30],
    'model__min_samples_split': [2, 5],
    'model__min_samples_leaf': [1, 2]
}

# Siapkan GridSearchCV untuk dijalankan pada SELURUH pipeline
grid_search = GridSearchCV(estimator=full_pipeline, param_grid=param_grid_rf, cv=3, n_jobs=-1, scoring='r2', verbose=2)

print("\nMemulai pencarian parameter terbaik untuk Random Forest di dalam pipeline...")
start_time = time.time()
grid_search.fit(X_train, y_train)
training_time = time.time() - start_time


# Step 5: Menampilkan Hasil Evaluasi
# Ambil pipeline terbaik yang ditemukan
best_pipeline = grid_search.best_estimator_

# Lakukan prediksi menggunakan pipeline terbaik
y_pred = best_pipeline.predict(X_test)

# Hitung semua metrik
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Tampilkan semua hasil
print("\nHASIL EVALUASI MODEL RANDOM FOREST")
print(f"Waktu training          : {training_time:.2f} detik")
print(f"Parameter terbaik        : {grid_search.best_params_}")
print("-" * 50)
print(f"R-squared (R²)           : {r2:.4f}")
print(f"Mean Absolute Error (MAE)  : {mae:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")


Target yang akan diprediksi: 'PENGAJUAN KE PLANNER_FULL_20 DC'

Membersihkan kolom fitur numerik...
  - Kolom 'Voyage No.' telah dipastikan menjadi numerik.
  - Kolom 'Voyage Yr' telah dipastikan menjadi numerik.
  - Kolom 'TOTAL BONGKARAN_FULL_20 DC' telah dipastikan menjadi numerik.

Pipeline untuk preprocessing dan model Random Forest berhasil dibuat.

Data dibagi menjadi:
  - 1076 baris untuk training
  - 270 baris untuk testing

Memulai pencarian parameter terbaik untuk Random Forest di dalam pipeline...
Fitting 3 folds for each of 24 candidates, totalling 72 fits

HASIL EVALUASI MODEL RANDOM FOREST
Waktu training          : 25.25 detik
Parameter terbaik        : {'model__max_depth': 10, 'model__min_samples_leaf': 2, 'model__min_samples_split': 2, 'model__n_estimators': 200}
--------------------------------------------------
R-squared (R²)           : 0.9258
Mean Absolute Error (MAE)  : 1.7691
Root Mean Squared Error (RMSE): 14.7068


In [21]:
import joblib

joblib.dump(best_pipeline, 'bongkaran_pengajuan_full_20.pkl')

['bongkaran_pengajuan_full_20.pkl']

# Bongkaran to Pengajuan Full 40 HC

In [22]:
# Step 1: Import Semua Library yang Dibutuhkan
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

# Library untuk Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder


# Step 2: Membaca dan Membersihkan Data Awal
# Membaca file
df = pd.read_excel('Ship_Operation_Data_Cleaned_1.xlsx')

# Tentukan fitur dan target
kolom_fitur_kategorikal = ['VESSEL ID (DMY)', 'BERTH LOCATION']
kolom_fitur_numerik = ['Voyage No.', 'Voyage Yr', 'TOTAL BONGKARAN_FULL_40 HC']
kolom_fitur = kolom_fitur_kategorikal + kolom_fitur_numerik
kolom_target = 'PENGAJUAN KE PLANNER_FULL_40 HC'

print(f"\nTarget yang akan diprediksi: '{kolom_target}'")

print("\nMembersihkan kolom fitur numerik...")
for col in kolom_fitur_numerik:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    print(f"  - Kolom '{col}' telah dipastikan menjadi numerik.")

# Membersihkan data di kolom target (fitur akan ditangani oleh Pipeline)
df[kolom_target] = pd.to_numeric(df[kolom_target], errors='coerce')
if df[kolom_target].isnull().sum() > 0:
    median_value = df[kolom_target].median()
    df[kolom_target].fillna(median_value, inplace=True)
    print(f"  - Nilai kosong di '{kolom_target}' diisi dengan median ({median_value}).")


# Step 3: Membuat Pipeline Preprocessing dan Model
# Definisikan untuk kolom numerik: isi nilai kosong dengan median.
numeric_transformer = SimpleImputer(strategy='median')

# Definisikan untuk kolom kategorikal: isi nilai kosong lalu lakukan one-hot encoding.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')) # 'ignore' agar tidak error jika ada kategori baru saat prediksi
])

# Gabungkan kedua di atas menjadi satu preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, kolom_fitur_numerik),
        ('cat', categorical_transformer, kolom_fitur_kategorikal)
    ])

# Buat Pipeline Lengkap: rangkai preprocessor dengan model Random Forest
full_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', RandomForestRegressor(random_state=42))
])

print("\nPipeline untuk preprocessing dan model Random Forest berhasil dibuat.")


# Step 4: Membagi Data dan Melakukan GridSearchCV pada Pipeline
# Pisahkan fitur (X) dan target (y)
X = df[kolom_fitur]
y = df[kolom_target]

# Bagi data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nData dibagi menjadi:")
print(f"  - {X_train.shape[0]} baris untuk training")
print(f"  - {X_test.shape[0]} baris untuk testing")

# Tentukan parameter yang ingin diuji untuk model di dalam pipeline
# PENTING: nama parameter harus diawali dengan nama langkah di pipeline, yaitu 'model__'
param_grid_rf = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [10, 20, 30],
    'model__min_samples_split': [2, 5],
    'model__min_samples_leaf': [1, 2]
}

# Siapkan GridSearchCV untuk dijalankan pada SELURUH pipeline
grid_search = GridSearchCV(estimator=full_pipeline, param_grid=param_grid_rf, cv=3, n_jobs=-1, scoring='r2', verbose=2)

print("\nMemulai pencarian parameter terbaik untuk Random Forest di dalam pipeline...")
start_time = time.time()
grid_search.fit(X_train, y_train)
training_time = time.time() - start_time


# Step 5: Menampilkan Hasil Evaluasi
# Ambil pipeline terbaik yang ditemukan
best_pipeline = grid_search.best_estimator_

# Lakukan prediksi menggunakan pipeline terbaik
y_pred = best_pipeline.predict(X_test)

# Hitung semua metrik
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Tampilkan semua hasil
print("\nHASIL EVALUASI MODEL RANDOM FOREST")
print(f"Waktu training          : {training_time:.2f} detik")
print(f"Parameter terbaik        : {grid_search.best_params_}")
print("-" * 50)
print(f"R-squared (R²)           : {r2:.4f}")
print(f"Mean Absolute Error (MAE)  : {mae:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")


Target yang akan diprediksi: 'PENGAJUAN KE PLANNER_FULL_40 HC'

Membersihkan kolom fitur numerik...
  - Kolom 'Voyage No.' telah dipastikan menjadi numerik.
  - Kolom 'Voyage Yr' telah dipastikan menjadi numerik.
  - Kolom 'TOTAL BONGKARAN_FULL_40 HC' telah dipastikan menjadi numerik.

Pipeline untuk preprocessing dan model Random Forest berhasil dibuat.

Data dibagi menjadi:
  - 1076 baris untuk training
  - 270 baris untuk testing

Memulai pencarian parameter terbaik untuk Random Forest di dalam pipeline...
Fitting 3 folds for each of 24 candidates, totalling 72 fits

HASIL EVALUASI MODEL RANDOM FOREST
Waktu training          : 23.26 detik
Parameter terbaik        : {'model__max_depth': 10, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 200}
--------------------------------------------------
R-squared (R²)           : 0.7986
Mean Absolute Error (MAE)  : 0.6293
Root Mean Squared Error (RMSE): 5.2857


In [23]:
import joblib

joblib.dump(best_pipeline, 'bongkaran_pengajuan_full_40.pkl')

['bongkaran_pengajuan_full_40.pkl']

# Pengajuan to ACC Empty 20 DC

In [33]:
# Step 1: Import Semua Library yang Dibutuhkan
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

# Library untuk Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder


# Step 2: Membaca dan Membersihkan Data Awal
# Membaca file
df = pd.read_excel('Ship_Operation_Data_Cleaned_1.xlsx')

# Tentukan fitur dan target
kolom_fitur_kategorikal = ['VESSEL ID (DMY)', 'BERTH LOCATION']
kolom_fitur_numerik = ['Voyage No.', 'Voyage Yr', 'TOTAL BONGKARAN_EMPTY_20 DC',
                       'PENGAJUAN KE PLANNER_EMPTY_20 DC']
kolom_fitur = kolom_fitur_kategorikal + kolom_fitur_numerik
kolom_target = 'ACC PENGAJUAN_EMPTY_20 DC'

print(f"\nTarget yang akan diprediksi: '{kolom_target}'")

print("\nMembersihkan kolom fitur numerik...")
for col in kolom_fitur_numerik:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    print(f"  - Kolom '{col}' telah dipastikan menjadi numerik.")

# Membersihkan data di kolom target (fitur akan ditangani oleh Pipeline)
df[kolom_target] = pd.to_numeric(df[kolom_target], errors='coerce')
if df[kolom_target].isnull().sum() > 0:
    median_value = df[kolom_target].median()
    df[kolom_target].fillna(median_value, inplace=True)
    print(f"  - Nilai kosong di '{kolom_target}' diisi dengan median ({median_value}).")


# Step 3: Membuat Pipeline Preprocessing dan Model
# Definisikan untuk kolom numerik: isi nilai kosong dengan median.
numeric_transformer = SimpleImputer(strategy='median')

# Definisikan untuk kolom kategorikal: isi nilai kosong lalu lakukan one-hot encoding.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')) # 'ignore' agar tidak error jika ada kategori baru saat prediksi
])

# Gabungkan kedua di atas menjadi satu preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, kolom_fitur_numerik),
        ('cat', categorical_transformer, kolom_fitur_kategorikal)
    ])

# Buat Pipeline Lengkap: rangkai preprocessor dengan model Random Forest
full_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', RandomForestRegressor(random_state=42))
])

print("\nPipeline untuk preprocessing dan model Random Forest berhasil dibuat.")


# Step 4: Membagi Data dan Melakukan GridSearchCV pada Pipeline
# Pisahkan fitur (X) dan target (y)
X = df[kolom_fitur]
y = df[kolom_target]

# Bagi data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nData dibagi menjadi:")
print(f"  - {X_train.shape[0]} baris untuk training")
print(f"  - {X_test.shape[0]} baris untuk testing")

# Tentukan parameter yang ingin diuji untuk model di dalam pipeline
# PENTING: nama parameter harus diawali dengan nama langkah di pipeline, yaitu 'model__'
param_grid_rf = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [10, 20, 30],
    'model__min_samples_split': [2, 5],
    'model__min_samples_leaf': [1, 2]
}

# Siapkan GridSearchCV untuk dijalankan pada SELURUH pipeline
grid_search = GridSearchCV(estimator=full_pipeline, param_grid=param_grid_rf, cv=3, n_jobs=-1, scoring='r2', verbose=2)

print("\nMemulai pencarian parameter terbaik untuk Random Forest di dalam pipeline...")
start_time = time.time()
grid_search.fit(X_train, y_train)
training_time = time.time() - start_time


# Step 5: Menampilkan Hasil Evaluasi
# Ambil pipeline terbaik yang ditemukan
best_pipeline = grid_search.best_estimator_

# Lakukan prediksi menggunakan pipeline terbaik
y_pred = best_pipeline.predict(X_test)

# Hitung semua metrik
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Tampilkan semua hasil
print("\nHASIL EVALUASI MODEL RANDOM FOREST")
print(f"Waktu training          : {training_time:.2f} detik")
print(f"Parameter terbaik        : {grid_search.best_params_}")
print("-" * 50)
print(f"R-squared (R²)           : {r2:.4f}")
print(f"Mean Absolute Error (MAE)  : {mae:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")


Target yang akan diprediksi: 'ACC PENGAJUAN_EMPTY_20 DC'

Membersihkan kolom fitur numerik...
  - Kolom 'Voyage No.' telah dipastikan menjadi numerik.
  - Kolom 'Voyage Yr' telah dipastikan menjadi numerik.
  - Kolom 'TOTAL BONGKARAN_EMPTY_20 DC' telah dipastikan menjadi numerik.
  - Kolom 'PENGAJUAN KE PLANNER_EMPTY_20 DC' telah dipastikan menjadi numerik.

Pipeline untuk preprocessing dan model Random Forest berhasil dibuat.

Data dibagi menjadi:
  - 1076 baris untuk training
  - 270 baris untuk testing

Memulai pencarian parameter terbaik untuk Random Forest di dalam pipeline...
Fitting 3 folds for each of 24 candidates, totalling 72 fits

HASIL EVALUASI MODEL RANDOM FOREST
Waktu training          : 46.33 detik
Parameter terbaik        : {'model__max_depth': 20, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 200}
--------------------------------------------------
R-squared (R²)           : 0.9297
Mean Absolute Error (MAE)  : 8.6761
Root Mean Squ

In [34]:
import joblib

joblib.dump(best_pipeline, 'pengajuan_acc_empty_20.pkl')

['pengajuan_acc_empty_20.pkl']

# Pengajuan to ACC Empty 40 HC

In [26]:
# Step 1: Import Semua Library yang Dibutuhkan
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

# Library untuk Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder


# Step 2: Membaca dan Membersihkan Data Awal
# Membaca file
df = pd.read_excel('Ship_Operation_Data_Cleaned_1.xlsx')

# Tentukan fitur dan target
kolom_fitur_kategorikal = ['VESSEL ID (DMY)', 'BERTH LOCATION']
kolom_fitur_numerik = ['Voyage No.', 'Voyage Yr', 'TOTAL BONGKARAN_EMPTY_40 HC',
                       'PENGAJUAN KE PLANNER_EMPTY_40 HC']
kolom_fitur = kolom_fitur_kategorikal + kolom_fitur_numerik
kolom_target = 'ACC PENGAJUAN_EMPTY_40 HC'

print(f"\nTarget yang akan diprediksi: '{kolom_target}'")

print("\nMembersihkan kolom fitur numerik...")
for col in kolom_fitur_numerik:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    print(f"  - Kolom '{col}' telah dipastikan menjadi numerik.")

# Membersihkan data di kolom target (fitur akan ditangani oleh Pipeline)
df[kolom_target] = pd.to_numeric(df[kolom_target], errors='coerce')
if df[kolom_target].isnull().sum() > 0:
    median_value = df[kolom_target].median()
    df[kolom_target].fillna(median_value, inplace=True)
    print(f"  - Nilai kosong di '{kolom_target}' diisi dengan median ({median_value}).")


# Step 3: Membuat Pipeline Preprocessing dan Model
# Definisikan untuk kolom numerik: isi nilai kosong dengan median.
numeric_transformer = SimpleImputer(strategy='median')

# Definisikan untuk kolom kategorikal: isi nilai kosong lalu lakukan one-hot encoding.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')) # 'ignore' agar tidak error jika ada kategori baru saat prediksi
])

# Gabungkan kedua di atas menjadi satu preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, kolom_fitur_numerik),
        ('cat', categorical_transformer, kolom_fitur_kategorikal)
    ])

# Buat Pipeline Lengkap: rangkai preprocessor dengan model Random Forest
full_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', RandomForestRegressor(random_state=42))
])

print("\nPipeline untuk preprocessing dan model Random Forest berhasil dibuat.")


# Step 4: Membagi Data dan Melakukan GridSearchCV pada Pipeline
# Pisahkan fitur (X) dan target (y)
X = df[kolom_fitur]
y = df[kolom_target]

# Bagi data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nData dibagi menjadi:")
print(f"  - {X_train.shape[0]} baris untuk training")
print(f"  - {X_test.shape[0]} baris untuk testing")

# Tentukan parameter yang ingin diuji untuk model di dalam pipeline
# PENTING: nama parameter harus diawali dengan nama langkah di pipeline, yaitu 'model__'
param_grid_rf = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [10, 20, 30],
    'model__min_samples_split': [2, 5],
    'model__min_samples_leaf': [1, 2]
}

# Siapkan GridSearchCV untuk dijalankan pada SELURUH pipeline
grid_search = GridSearchCV(estimator=full_pipeline, param_grid=param_grid_rf, cv=3, n_jobs=-1, scoring='r2', verbose=2)

print("\nMemulai pencarian parameter terbaik untuk Random Forest di dalam pipeline...")
start_time = time.time()
grid_search.fit(X_train, y_train)
training_time = time.time() - start_time


# Step 5: Menampilkan Hasil Evaluasi
# Ambil pipeline terbaik yang ditemukan
best_pipeline = grid_search.best_estimator_

# Lakukan prediksi menggunakan pipeline terbaik
y_pred = best_pipeline.predict(X_test)

# Hitung semua metrik
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Tampilkan semua hasil
print("\nHASIL EVALUASI MODEL RANDOM FOREST")
print(f"Waktu training          : {training_time:.2f} detik")
print(f"Parameter terbaik        : {grid_search.best_params_}")
print("-" * 50)
print(f"R-squared (R²)           : {r2:.4f}")
print(f"Mean Absolute Error (MAE)  : {mae:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")


Target yang akan diprediksi: 'ACC PENGAJUAN_EMPTY_40 HC'

Membersihkan kolom fitur numerik...
  - Kolom 'Voyage No.' telah dipastikan menjadi numerik.
  - Kolom 'Voyage Yr' telah dipastikan menjadi numerik.
  - Kolom 'TOTAL BONGKARAN_EMPTY_40 HC' telah dipastikan menjadi numerik.
  - Kolom 'PENGAJUAN KE PLANNER_EMPTY_40 HC' telah dipastikan menjadi numerik.

Pipeline untuk preprocessing dan model Random Forest berhasil dibuat.

Data dibagi menjadi:
  - 1076 baris untuk training
  - 270 baris untuk testing

Memulai pencarian parameter terbaik untuk Random Forest di dalam pipeline...
Fitting 3 folds for each of 24 candidates, totalling 72 fits

HASIL EVALUASI MODEL RANDOM FOREST
Waktu training          : 41.58 detik
Parameter terbaik        : {'model__max_depth': 10, 'model__min_samples_leaf': 2, 'model__min_samples_split': 5, 'model__n_estimators': 100}
--------------------------------------------------
R-squared (R²)           : 0.8414
Mean Absolute Error (MAE)  : 3.4209
Root Mean Squ

In [27]:
import joblib

joblib.dump(best_pipeline, 'pengajuan_acc_empty_40.pkl')

['pengajuan_acc_empty_40.pkl']

# Pengajuan to ACC Full 20 DC

In [28]:
# Step 1: Import Semua Library yang Dibutuhkan
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

# Library untuk Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder


# Step 2: Membaca dan Membersihkan Data Awal
# Membaca file
df = pd.read_excel('Ship_Operation_Data_Cleaned_1.xlsx')

# Tentukan fitur dan target
kolom_fitur_kategorikal = ['VESSEL ID (DMY)', 'BERTH LOCATION']
kolom_fitur_numerik = ['Voyage No.', 'Voyage Yr', 'TOTAL BONGKARAN_FULL_20 DC',
                       'PENGAJUAN KE PLANNER_FULL_20 DC']
kolom_fitur = kolom_fitur_kategorikal + kolom_fitur_numerik
kolom_target = 'ACC PENGAJUAN_FULL_20 DC'

print(f"\nTarget yang akan diprediksi: '{kolom_target}'")

print("\nMembersihkan kolom fitur numerik...")
for col in kolom_fitur_numerik:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    print(f"  - Kolom '{col}' telah dipastikan menjadi numerik.")

# Membersihkan data di kolom target (fitur akan ditangani oleh Pipeline)
df[kolom_target] = pd.to_numeric(df[kolom_target], errors='coerce')
if df[kolom_target].isnull().sum() > 0:
    median_value = df[kolom_target].median()
    df[kolom_target].fillna(median_value, inplace=True)
    print(f"  - Nilai kosong di '{kolom_target}' diisi dengan median ({median_value}).")


# Step 3: Membuat Pipeline Preprocessing dan Model
# Definisikan untuk kolom numerik: isi nilai kosong dengan median.
numeric_transformer = SimpleImputer(strategy='median')

# Definisikan untuk kolom kategorikal: isi nilai kosong lalu lakukan one-hot encoding.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')) # 'ignore' agar tidak error jika ada kategori baru saat prediksi
])

# Gabungkan kedua di atas menjadi satu preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, kolom_fitur_numerik),
        ('cat', categorical_transformer, kolom_fitur_kategorikal)
    ])

# Buat Pipeline Lengkap: rangkai preprocessor dengan model Random Forest
full_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', RandomForestRegressor(random_state=42))
])

print("\nPipeline untuk preprocessing dan model Random Forest berhasil dibuat.")


# Step 4: Membagi Data dan Melakukan GridSearchCV pada Pipeline
# Pisahkan fitur (X) dan target (y)
X = df[kolom_fitur]
y = df[kolom_target]

# Bagi data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nData dibagi menjadi:")
print(f"  - {X_train.shape[0]} baris untuk training")
print(f"  - {X_test.shape[0]} baris untuk testing")

# Tentukan parameter yang ingin diuji untuk model di dalam pipeline
# PENTING: nama parameter harus diawali dengan nama langkah di pipeline, yaitu 'model__'
param_grid_rf = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [10, 20, 30],
    'model__min_samples_split': [2, 5],
    'model__min_samples_leaf': [1, 2]
}

# Siapkan GridSearchCV untuk dijalankan pada SELURUH pipeline
grid_search = GridSearchCV(estimator=full_pipeline, param_grid=param_grid_rf, cv=3, n_jobs=-1, scoring='r2', verbose=2)

print("\nMemulai pencarian parameter terbaik untuk Random Forest di dalam pipeline...")
start_time = time.time()
grid_search.fit(X_train, y_train)
training_time = time.time() - start_time


# Step 5: Menampilkan Hasil Evaluasi
# Ambil pipeline terbaik yang ditemukan
best_pipeline = grid_search.best_estimator_

# Lakukan prediksi menggunakan pipeline terbaik
y_pred = best_pipeline.predict(X_test)

# Hitung semua metrik
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Tampilkan semua hasil
print("\nHASIL EVALUASI MODEL RANDOM FOREST")
print(f"Waktu training          : {training_time:.2f} detik")
print(f"Parameter terbaik        : {grid_search.best_params_}")
print("-" * 50)
print(f"R-squared (R²)           : {r2:.4f}")
print(f"Mean Absolute Error (MAE)  : {mae:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")


Target yang akan diprediksi: 'ACC PENGAJUAN_FULL_20 DC'

Membersihkan kolom fitur numerik...
  - Kolom 'Voyage No.' telah dipastikan menjadi numerik.
  - Kolom 'Voyage Yr' telah dipastikan menjadi numerik.
  - Kolom 'TOTAL BONGKARAN_FULL_20 DC' telah dipastikan menjadi numerik.
  - Kolom 'PENGAJUAN KE PLANNER_FULL_20 DC' telah dipastikan menjadi numerik.

Pipeline untuk preprocessing dan model Random Forest berhasil dibuat.

Data dibagi menjadi:
  - 1076 baris untuk training
  - 270 baris untuk testing

Memulai pencarian parameter terbaik untuk Random Forest di dalam pipeline...
Fitting 3 folds for each of 24 candidates, totalling 72 fits

HASIL EVALUASI MODEL RANDOM FOREST
Waktu training          : 29.20 detik
Parameter terbaik        : {'model__max_depth': 20, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 200}
--------------------------------------------------
R-squared (R²)           : 0.9314
Mean Absolute Error (MAE)  : 1.2578
Root Mean Square

In [29]:
import joblib

joblib.dump(best_pipeline, 'pengajuan_acc_full_20.pkl')

['pengajuan_acc_full_20.pkl']

# Pengajuan to ACC Full 40 HC

In [30]:
# Step 1: Import Semua Library yang Dibutuhkan
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

# Library untuk Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder


# Step 2: Membaca dan Membersihkan Data Awal
# Membaca file
df = pd.read_excel('Ship_Operation_Data_Cleaned_1.xlsx')

# Tentukan fitur dan target
kolom_fitur_kategorikal = ['VESSEL ID (DMY)', 'BERTH LOCATION']
kolom_fitur_numerik = ['Voyage No.', 'Voyage Yr', 'TOTAL BONGKARAN_FULL_40 HC',
                       'PENGAJUAN KE PLANNER_FULL_40 HC']
kolom_fitur = kolom_fitur_kategorikal + kolom_fitur_numerik
kolom_target = 'ACC PENGAJUAN_FULL_40 HC'

print(f"\nTarget yang akan diprediksi: '{kolom_target}'")

print("\nMembersihkan kolom fitur numerik...")
for col in kolom_fitur_numerik:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    print(f"  - Kolom '{col}' telah dipastikan menjadi numerik.")

# Membersihkan data di kolom target (fitur akan ditangani oleh Pipeline)
df[kolom_target] = pd.to_numeric(df[kolom_target], errors='coerce')
if df[kolom_target].isnull().sum() > 0:
    median_value = df[kolom_target].median()
    df[kolom_target].fillna(median_value, inplace=True)
    print(f"  - Nilai kosong di '{kolom_target}' diisi dengan median ({median_value}).")


# Step 3: Membuat Pipeline Preprocessing dan Model
# Definisikan untuk kolom numerik: isi nilai kosong dengan median.
numeric_transformer = SimpleImputer(strategy='median')

# Definisikan untuk kolom kategorikal: isi nilai kosong lalu lakukan one-hot encoding.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')) # 'ignore' agar tidak error jika ada kategori baru saat prediksi
])

# Gabungkan kedua di atas menjadi satu preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, kolom_fitur_numerik),
        ('cat', categorical_transformer, kolom_fitur_kategorikal)
    ])

# Buat Pipeline Lengkap: rangkai preprocessor dengan model Random Forest
full_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', RandomForestRegressor(random_state=42))
])

print("\nPipeline untuk preprocessing dan model Random Forest berhasil dibuat.")


# Step 4: Membagi Data dan Melakukan GridSearchCV pada Pipeline
# Pisahkan fitur (X) dan target (y)
X = df[kolom_fitur]
y = df[kolom_target]

# Bagi data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nData dibagi menjadi:")
print(f"  - {X_train.shape[0]} baris untuk training")
print(f"  - {X_test.shape[0]} baris untuk testing")

# Tentukan parameter yang ingin diuji untuk model di dalam pipeline
# PENTING: nama parameter harus diawali dengan nama langkah di pipeline, yaitu 'model__'
param_grid_rf = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [10, 20, 30],
    'model__min_samples_split': [2, 5],
    'model__min_samples_leaf': [1, 2]
}

# Siapkan GridSearchCV untuk dijalankan pada SELURUH pipeline
grid_search = GridSearchCV(estimator=full_pipeline, param_grid=param_grid_rf, cv=3, n_jobs=-1, scoring='r2', verbose=2)

print("\nMemulai pencarian parameter terbaik untuk Random Forest di dalam pipeline...")
start_time = time.time()
grid_search.fit(X_train, y_train)
training_time = time.time() - start_time


# Step 5: Menampilkan Hasil Evaluasi
# Ambil pipeline terbaik yang ditemukan
best_pipeline = grid_search.best_estimator_

# Lakukan prediksi menggunakan pipeline terbaik
y_pred = best_pipeline.predict(X_test)

# Hitung semua metrik
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Tampilkan semua hasil
print("\nHASIL EVALUASI MODEL RANDOM FOREST")
print(f"Waktu training          : {training_time:.2f} detik")
print(f"Parameter terbaik        : {grid_search.best_params_}")
print("-" * 50)
print(f"R-squared (R²)           : {r2:.4f}")
print(f"Mean Absolute Error (MAE)  : {mae:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")


Target yang akan diprediksi: 'ACC PENGAJUAN_FULL_40 HC'

Membersihkan kolom fitur numerik...
  - Kolom 'Voyage No.' telah dipastikan menjadi numerik.
  - Kolom 'Voyage Yr' telah dipastikan menjadi numerik.
  - Kolom 'TOTAL BONGKARAN_FULL_40 HC' telah dipastikan menjadi numerik.
  - Kolom 'PENGAJUAN KE PLANNER_FULL_40 HC' telah dipastikan menjadi numerik.
  - Nilai kosong di 'ACC PENGAJUAN_FULL_40 HC' diisi dengan median (0.0).

Pipeline untuk preprocessing dan model Random Forest berhasil dibuat.

Data dibagi menjadi:
  - 1076 baris untuk training
  - 270 baris untuk testing

Memulai pencarian parameter terbaik untuk Random Forest di dalam pipeline...
Fitting 3 folds for each of 24 candidates, totalling 72 fits


/tmp/ipython-input-930299307.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[kolom_target].fillna(median_value, inplace=True)



HASIL EVALUASI MODEL RANDOM FOREST
Waktu training          : 22.00 detik
Parameter terbaik        : {'model__max_depth': 10, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 200}
--------------------------------------------------
R-squared (R²)           : 0.7090
Mean Absolute Error (MAE)  : 0.6391
Root Mean Squared Error (RMSE): 5.8181


In [32]:
import joblib

joblib.dump(best_pipeline, 'pengajuan_acc_full_40.pkl')

['pengajuan_acc_full_40.pkl']

# ACC to Realisasi Shipside Empty 20 DC

In [39]:
# Step 1: Import Semua Library yang Dibutuhkan
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

# Library untuk Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder


# Step 2: Membaca dan Membersihkan Data Awal
# Membaca file
df = pd.read_excel('Ship_Operation_Data_Cleaned_1.xlsx')

kolom_fitur_kategorikal = ['VESSEL ID (DMY)', 'BERTH LOCATION']
kolom_fitur_numerik = [
    'Voyage No.', 'Voyage Yr', 'TOTAL BONGKARAN_EMPTY_20 DC',
    'PENGAJUAN KE PLANNER_EMPTY_20 DC', 'ACC PENGAJUAN_EMPTY_20 DC'
]
kolom_fitur = kolom_fitur_kategorikal + kolom_fitur_numerik
kolom_target = [
    'REALISASI ALL DEPO_ALL DEPO_MXD_20 DC', 'SHIPSIDE_YES_MXD_20 DC',
    'SHIPSIDE_NO_MXD_20 DC'
]

print(f"\nFitur yang digunakan: {kolom_fitur}")
print(f"Target yang akan diprediksi: {kolom_target}")


# Membersihkan semua kolom numerik (baik fitur maupun target)
print("\nMembersihkan semua kolom numerik...")
kolom_numerik_total = kolom_fitur_numerik + kolom_target
for col in kolom_numerik_total:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Hapus baris di mana targetnya kosong (karena tidak bisa untuk training)
df.dropna(subset=kolom_target, inplace=True)
print("Baris dengan nilai target kosong telah dihapus.")


# Step 3: Membuat Pipeline Preprocessing dan Model (TIDAK ADA PERUBAHAN)
# Definisikan untuk kolom numerik: isi nilai kosong dengan median.
numeric_transformer = SimpleImputer(strategy='median')

# Definisikan untuk kolom kategorikal: isi nilai kosong lalu lakukan one-hot encoding.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Gabungkan kedua di atas menjadi satu preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, kolom_fitur_numerik),
        ('cat', categorical_transformer, kolom_fitur_kategorikal)
    ])

# Buat Pipeline Lengkap
full_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', RandomForestRegressor(random_state=42))
])

print("\nPipeline untuk preprocessing dan model Random Forest berhasil dibuat.")


# Step 4: Membagi Data dan Melakukan GridSearchCV pada Pipeline
X = df[kolom_fitur]
y = df[kolom_target] # y sekarang berisi 3 kolom

# Bagi data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nData dibagi menjadi:")
print(f"  - {X_train.shape[0]} baris untuk training")
print(f"  - {X_test.shape[0]} baris untuk testing")

# Tentukan parameter yang ingin diuji (tidak perlu diubah)
param_grid_rf = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [10, 20, 30],
    'model__min_samples_split': [2, 5],
    'model__min_samples_leaf': [1, 2]

}

# Siapkan GridSearchCV (tidak perlu diubah)
grid_search = GridSearchCV(estimator=full_pipeline, param_grid=param_grid_rf, cv=3, n_jobs=-1, scoring='r2', verbose=2)

print("\nMemulai pencarian parameter terbaik untuk Multi-output...")
start_time = time.time()
grid_search.fit(X_train, y_train)
training_time = time.time() - start_time


# Step 5: Menampilkan Hasil Evaluasi
best_pipeline = grid_search.best_estimator_
y_pred = best_pipeline.predict(X_test)

# === PERUBAHAN 3: CARA MENAMPILKAN EVALUASI METRIK ===
print("\nHASIL EVALUASI MODEL MULTI-OUTPUT")
print(f"Waktu training          : {training_time:.2f} detik")
print(f"Parameter terbaik        : {grid_search.best_params_}")

# Menghitung R2, MAE, dan RMSE
# 'uniform_average' -> merata-ratakan skor dari ketiga target
# 'raw_values' -> menampilkan skor untuk masing-masing target
r2_avg = r2_score(y_test, y_pred, multioutput='uniform_average')
mae_avg = mean_absolute_error(y_test, y_pred, multioutput='uniform_average')
rmse_avg = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='uniform_average'))

r2_raw = r2_score(y_test, y_pred, multioutput='raw_values')
mae_raw = mean_absolute_error(y_test, y_pred, multioutput='raw_values')
rmse_raw = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))


print(f"R-squared (R²) Rata-rata           : {r2_avg:.4f}")
print(f"Mean Absolute Error (MAE) Rata-rata  : {mae_avg:.4f}")
print(f"Root Mean Squared Error (RMSE) Rata-rata: {rmse_avg:.4f}")
print("-" * 50)
print("SKOR UNTUK MASING-MASING TARGET:")
for i, target_name in enumerate(kolom_target):
    print(f"  -> Target: {target_name}")
    print(f"     R²  : {r2_raw[i]:.4f}")
    print(f"     MAE : {mae_raw[i]:.4f}")
    print(f"     RMSE: {rmse_raw[i]:.4f}")


Fitur yang digunakan: ['VESSEL ID (DMY)', 'BERTH LOCATION', 'Voyage No.', 'Voyage Yr', 'TOTAL BONGKARAN_EMPTY_20 DC', 'PENGAJUAN KE PLANNER_EMPTY_20 DC', 'ACC PENGAJUAN_EMPTY_20 DC']
Target yang akan diprediksi: ['REALISASI ALL DEPO_ALL DEPO_MXD_20 DC', 'SHIPSIDE_YES_MXD_20 DC', 'SHIPSIDE_NO_MXD_20 DC']

Membersihkan semua kolom numerik...
Baris dengan nilai target kosong telah dihapus.

Pipeline untuk preprocessing dan model Random Forest berhasil dibuat.

Data dibagi menjadi:
  - 1076 baris untuk training
  - 270 baris untuk testing

Memulai pencarian parameter terbaik untuk Multi-output...
Fitting 3 folds for each of 24 candidates, totalling 72 fits

HASIL EVALUASI MODEL MULTI-OUTPUT
Waktu training          : 48.55 detik
Parameter terbaik        : {'model__max_depth': 10, 'model__min_samples_leaf': 2, 'model__min_samples_split': 5, 'model__n_estimators': 100}
R-squared (R²) Rata-rata           : 0.4118
Mean Absolute Error (MAE) Rata-rata  : 7.2160
Root Mean Squared Error (RMSE) Rat

In [40]:
import joblib

joblib.dump(best_pipeline, 'acc_realisasi_shipside_empty_20.pkl')

['acc_realisasi_shipside_empty_20.pkl']

# ACC to Realisasi Shipside Empty 40 HC

In [41]:
# Step 1: Import Semua Library yang Dibutuhkan
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

# Library untuk Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder


# Step 2: Membaca dan Membersihkan Data Awal
# Membaca file
df = pd.read_excel('Ship_Operation_Data_Cleaned_1.xlsx')

kolom_fitur_kategorikal = ['VESSEL ID (DMY)', 'BERTH LOCATION']
kolom_fitur_numerik = [
    'Voyage No.', 'Voyage Yr', 'TOTAL BONGKARAN_EMPTY_40 HC',
    'PENGAJUAN KE PLANNER_EMPTY_40 HC', 'ACC PENGAJUAN_EMPTY_40 HC'
]
kolom_fitur = kolom_fitur_kategorikal + kolom_fitur_numerik
kolom_target = [
    'REALISASI ALL DEPO_ALL DEPO_MXD_40 HC', 'SHIPSIDE_YES_MXD_40 HC',
    'SHIPSIDE_NO_MXD_40 HC'
]

print(f"\nFitur yang digunakan: {kolom_fitur}")
print(f"Target yang akan diprediksi: {kolom_target}")


# Membersihkan semua kolom numerik (baik fitur maupun target)
print("\nMembersihkan semua kolom numerik...")
kolom_numerik_total = kolom_fitur_numerik + kolom_target
for col in kolom_numerik_total:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Hapus baris di mana targetnya kosong (karena tidak bisa untuk training)
df.dropna(subset=kolom_target, inplace=True)
print("Baris dengan nilai target kosong telah dihapus.")


# Step 3: Membuat Pipeline Preprocessing dan Model (TIDAK ADA PERUBAHAN)
# Definisikan untuk kolom numerik: isi nilai kosong dengan median.
numeric_transformer = SimpleImputer(strategy='median')

# Definisikan untuk kolom kategorikal: isi nilai kosong lalu lakukan one-hot encoding.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Gabungkan kedua di atas menjadi satu preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, kolom_fitur_numerik),
        ('cat', categorical_transformer, kolom_fitur_kategorikal)
    ])

# Buat Pipeline Lengkap
full_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', RandomForestRegressor(random_state=42))
])

print("\nPipeline untuk preprocessing dan model Random Forest berhasil dibuat.")


# Step 4: Membagi Data dan Melakukan GridSearchCV pada Pipeline
X = df[kolom_fitur]
y = df[kolom_target] # y sekarang berisi 3 kolom

# Bagi data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nData dibagi menjadi:")
print(f"  - {X_train.shape[0]} baris untuk training")
print(f"  - {X_test.shape[0]} baris untuk testing")

# Tentukan parameter yang ingin diuji (tidak perlu diubah)
param_grid_rf = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [10, 20, 30],
    'model__min_samples_split': [2, 5],
    'model__min_samples_leaf': [1, 2]

}

# Siapkan GridSearchCV (tidak perlu diubah)
grid_search = GridSearchCV(estimator=full_pipeline, param_grid=param_grid_rf, cv=3, n_jobs=-1, scoring='r2', verbose=2)

print("\nMemulai pencarian parameter terbaik untuk Multi-output...")
start_time = time.time()
grid_search.fit(X_train, y_train)
training_time = time.time() - start_time


# Step 5: Menampilkan Hasil Evaluasi
best_pipeline = grid_search.best_estimator_
y_pred = best_pipeline.predict(X_test)

# === PERUBAHAN 3: CARA MENAMPILKAN EVALUASI METRIK ===
print("\nHASIL EVALUASI MODEL MULTI-OUTPUT")
print(f"Waktu training          : {training_time:.2f} detik")
print(f"Parameter terbaik        : {grid_search.best_params_}")

# Menghitung R2, MAE, dan RMSE
# 'uniform_average' -> merata-ratakan skor dari ketiga target
# 'raw_values' -> menampilkan skor untuk masing-masing target
r2_avg = r2_score(y_test, y_pred, multioutput='uniform_average')
mae_avg = mean_absolute_error(y_test, y_pred, multioutput='uniform_average')
rmse_avg = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='uniform_average'))

r2_raw = r2_score(y_test, y_pred, multioutput='raw_values')
mae_raw = mean_absolute_error(y_test, y_pred, multioutput='raw_values')
rmse_raw = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))


print(f"R-squared (R²) Rata-rata           : {r2_avg:.4f}")
print(f"Mean Absolute Error (MAE) Rata-rata  : {mae_avg:.4f}")
print(f"Root Mean Squared Error (RMSE) Rata-rata: {rmse_avg:.4f}")
print("-" * 50)
print("SKOR UNTUK MASING-MASING TARGET:")
for i, target_name in enumerate(kolom_target):
    print(f"  -> Target: {target_name}")
    print(f"     R²  : {r2_raw[i]:.4f}")
    print(f"     MAE : {mae_raw[i]:.4f}")
    print(f"     RMSE: {rmse_raw[i]:.4f}")


Fitur yang digunakan: ['VESSEL ID (DMY)', 'BERTH LOCATION', 'Voyage No.', 'Voyage Yr', 'TOTAL BONGKARAN_EMPTY_40 HC', 'PENGAJUAN KE PLANNER_EMPTY_40 HC', 'ACC PENGAJUAN_EMPTY_40 HC']
Target yang akan diprediksi: ['REALISASI ALL DEPO_ALL DEPO_MXD_40 HC', 'SHIPSIDE_YES_MXD_40 HC', 'SHIPSIDE_NO_MXD_40 HC']

Membersihkan semua kolom numerik...
Baris dengan nilai target kosong telah dihapus.

Pipeline untuk preprocessing dan model Random Forest berhasil dibuat.

Data dibagi menjadi:
  - 1076 baris untuk training
  - 270 baris untuk testing

Memulai pencarian parameter terbaik untuk Multi-output...
Fitting 3 folds for each of 24 candidates, totalling 72 fits

HASIL EVALUASI MODEL MULTI-OUTPUT
Waktu training          : 42.13 detik
Parameter terbaik        : {'model__max_depth': 10, 'model__min_samples_leaf': 2, 'model__min_samples_split': 5, 'model__n_estimators': 100}
R-squared (R²) Rata-rata           : 0.4517
Mean Absolute Error (MAE) Rata-rata  : 1.5952
Root Mean Squared Error (RMSE) Rat

In [43]:
import joblib

joblib.dump(best_pipeline, 'acc_realisasi_shipside_empty_40.pkl')

['acc_realisasi_shipside_empty_40.pkl']

# ACC to Realisasi Shipside Full 20 DC

In [44]:
# Step 1: Import Semua Library yang Dibutuhkan
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

# Library untuk Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder


# Step 2: Membaca dan Membersihkan Data Awal
# Membaca file
df = pd.read_excel('Ship_Operation_Data_Cleaned_1.xlsx')

kolom_fitur_kategorikal = ['VESSEL ID (DMY)', 'BERTH LOCATION']
kolom_fitur_numerik = [
    'Voyage No.', 'Voyage Yr', 'TOTAL BONGKARAN_FULL_20 DC',
    'PENGAJUAN KE PLANNER_FULL_20 DC', 'ACC PENGAJUAN_FULL_20 DC'
]
kolom_fitur = kolom_fitur_kategorikal + kolom_fitur_numerik
kolom_target = [
    'REALISASI ALL DEPO_ALL DEPO_FXD_20 DC', 'SHIPSIDE_YES_FXD_20 DC',
    'SHIPSIDE_NO_FXD_20 DC'
]

print(f"\nFitur yang digunakan: {kolom_fitur}")
print(f"Target yang akan diprediksi: {kolom_target}")


# Membersihkan semua kolom numerik (baik fitur maupun target)
print("\nMembersihkan semua kolom numerik...")
kolom_numerik_total = kolom_fitur_numerik + kolom_target
for col in kolom_numerik_total:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Hapus baris di mana targetnya kosong (karena tidak bisa untuk training)
df.dropna(subset=kolom_target, inplace=True)
print("Baris dengan nilai target kosong telah dihapus.")


# Step 3: Membuat Pipeline Preprocessing dan Model (TIDAK ADA PERUBAHAN)
# Definisikan untuk kolom numerik: isi nilai kosong dengan median.
numeric_transformer = SimpleImputer(strategy='median')

# Definisikan untuk kolom kategorikal: isi nilai kosong lalu lakukan one-hot encoding.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Gabungkan kedua di atas menjadi satu preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, kolom_fitur_numerik),
        ('cat', categorical_transformer, kolom_fitur_kategorikal)
    ])

# Buat Pipeline Lengkap
full_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', RandomForestRegressor(random_state=42))
])

print("\nPipeline untuk preprocessing dan model Random Forest berhasil dibuat.")


# Step 4: Membagi Data dan Melakukan GridSearchCV pada Pipeline
X = df[kolom_fitur]
y = df[kolom_target] # y sekarang berisi 3 kolom

# Bagi data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nData dibagi menjadi:")
print(f"  - {X_train.shape[0]} baris untuk training")
print(f"  - {X_test.shape[0]} baris untuk testing")

# Tentukan parameter yang ingin diuji (tidak perlu diubah)
param_grid_rf = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [10, 20, 30],
    'model__min_samples_split': [2, 5],
    'model__min_samples_leaf': [1, 2]

}

# Siapkan GridSearchCV (tidak perlu diubah)
grid_search = GridSearchCV(estimator=full_pipeline, param_grid=param_grid_rf, cv=3, n_jobs=-1, scoring='r2', verbose=2)

print("\nMemulai pencarian parameter terbaik untuk Multi-output...")
start_time = time.time()
grid_search.fit(X_train, y_train)
training_time = time.time() - start_time


# Step 5: Menampilkan Hasil Evaluasi
best_pipeline = grid_search.best_estimator_
y_pred = best_pipeline.predict(X_test)

# === PERUBAHAN 3: CARA MENAMPILKAN EVALUASI METRIK ===
print("\nHASIL EVALUASI MODEL MULTI-OUTPUT")
print(f"Waktu training          : {training_time:.2f} detik")
print(f"Parameter terbaik        : {grid_search.best_params_}")

# Menghitung R2, MAE, dan RMSE
# 'uniform_average' -> merata-ratakan skor dari ketiga target
# 'raw_values' -> menampilkan skor untuk masing-masing target
r2_avg = r2_score(y_test, y_pred, multioutput='uniform_average')
mae_avg = mean_absolute_error(y_test, y_pred, multioutput='uniform_average')
rmse_avg = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='uniform_average'))

r2_raw = r2_score(y_test, y_pred, multioutput='raw_values')
mae_raw = mean_absolute_error(y_test, y_pred, multioutput='raw_values')
rmse_raw = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))


print(f"R-squared (R²) Rata-rata           : {r2_avg:.4f}")
print(f"Mean Absolute Error (MAE) Rata-rata  : {mae_avg:.4f}")
print(f"Root Mean Squared Error (RMSE) Rata-rata: {rmse_avg:.4f}")
print("-" * 50)
print("SKOR UNTUK MASING-MASING TARGET:")
for i, target_name in enumerate(kolom_target):
    print(f"  -> Target: {target_name}")
    print(f"     R²  : {r2_raw[i]:.4f}")
    print(f"     MAE : {mae_raw[i]:.4f}")
    print(f"     RMSE: {rmse_raw[i]:.4f}")


Fitur yang digunakan: ['VESSEL ID (DMY)', 'BERTH LOCATION', 'Voyage No.', 'Voyage Yr', 'TOTAL BONGKARAN_FULL_20 DC', 'PENGAJUAN KE PLANNER_FULL_20 DC', 'ACC PENGAJUAN_FULL_20 DC']
Target yang akan diprediksi: ['REALISASI ALL DEPO_ALL DEPO_FXD_20 DC', 'SHIPSIDE_YES_FXD_20 DC', 'SHIPSIDE_NO_FXD_20 DC']

Membersihkan semua kolom numerik...
Baris dengan nilai target kosong telah dihapus.

Pipeline untuk preprocessing dan model Random Forest berhasil dibuat.

Data dibagi menjadi:
  - 1076 baris untuk training
  - 270 baris untuk testing

Memulai pencarian parameter terbaik untuk Multi-output...
Fitting 3 folds for each of 24 candidates, totalling 72 fits

HASIL EVALUASI MODEL MULTI-OUTPUT
Waktu training          : 33.26 detik
Parameter terbaik        : {'model__max_depth': 20, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 200}
R-squared (R²) Rata-rata           : 0.9925
Mean Absolute Error (MAE) Rata-rata  : 0.3731
Root Mean Squared Error (RMSE) Rata-r

In [45]:
import joblib

joblib.dump(best_pipeline, 'acc_realisasi_shipside_full_20.pkl')

['acc_realisasi_shipside_full_20.pkl']

# ACC to Realisasi Shipside Full 40 HC

In [46]:
# Step 1: Import Semua Library yang Dibutuhkan
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

# Library untuk Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder


# Step 2: Membaca dan Membersihkan Data Awal
# Membaca file
df = pd.read_excel('Ship_Operation_Data_Cleaned_1.xlsx')

kolom_fitur_kategorikal = ['VESSEL ID (DMY)', 'BERTH LOCATION']
kolom_fitur_numerik = [
    'Voyage No.', 'Voyage Yr', 'TOTAL BONGKARAN_FULL_40 HC',
    'PENGAJUAN KE PLANNER_FULL_40 HC', 'ACC PENGAJUAN_FULL_40 HC'
]
kolom_fitur = kolom_fitur_kategorikal + kolom_fitur_numerik
kolom_target = [
    'REALISASI ALL DEPO_ALL DEPO_FXD_40 HC', 'SHIPSIDE_YES_FXD_40 HC',
    'SHIPSIDE_NO_FXD_40 HC'
]

print(f"\nFitur yang digunakan: {kolom_fitur}")
print(f"Target yang akan diprediksi: {kolom_target}")


# Membersihkan semua kolom numerik (baik fitur maupun target)
print("\nMembersihkan semua kolom numerik...")
kolom_numerik_total = kolom_fitur_numerik + kolom_target
for col in kolom_numerik_total:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Hapus baris di mana targetnya kosong (karena tidak bisa untuk training)
df.dropna(subset=kolom_target, inplace=True)
print("Baris dengan nilai target kosong telah dihapus.")


# Step 3: Membuat Pipeline Preprocessing dan Model (TIDAK ADA PERUBAHAN)
# Definisikan untuk kolom numerik: isi nilai kosong dengan median.
numeric_transformer = SimpleImputer(strategy='median')

# Definisikan untuk kolom kategorikal: isi nilai kosong lalu lakukan one-hot encoding.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Gabungkan kedua di atas menjadi satu preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, kolom_fitur_numerik),
        ('cat', categorical_transformer, kolom_fitur_kategorikal)
    ])

# Buat Pipeline Lengkap
full_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', RandomForestRegressor(random_state=42))
])

print("\nPipeline untuk preprocessing dan model Random Forest berhasil dibuat.")


# Step 4: Membagi Data dan Melakukan GridSearchCV pada Pipeline
X = df[kolom_fitur]
y = df[kolom_target] # y sekarang berisi 3 kolom

# Bagi data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nData dibagi menjadi:")
print(f"  - {X_train.shape[0]} baris untuk training")
print(f"  - {X_test.shape[0]} baris untuk testing")

# Tentukan parameter yang ingin diuji (tidak perlu diubah)
param_grid_rf = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [10, 20, 30],
    'model__min_samples_split': [2, 5],
    'model__min_samples_leaf': [1, 2]

}

# Siapkan GridSearchCV (tidak perlu diubah)
grid_search = GridSearchCV(estimator=full_pipeline, param_grid=param_grid_rf, cv=3, n_jobs=-1, scoring='r2', verbose=2)

print("\nMemulai pencarian parameter terbaik untuk Multi-output...")
start_time = time.time()
grid_search.fit(X_train, y_train)
training_time = time.time() - start_time


# Step 5: Menampilkan Hasil Evaluasi
best_pipeline = grid_search.best_estimator_
y_pred = best_pipeline.predict(X_test)

# === PERUBAHAN 3: CARA MENAMPILKAN EVALUASI METRIK ===
print("\nHASIL EVALUASI MODEL MULTI-OUTPUT")
print(f"Waktu training          : {training_time:.2f} detik")
print(f"Parameter terbaik        : {grid_search.best_params_}")

# Menghitung R2, MAE, dan RMSE
# 'uniform_average' -> merata-ratakan skor dari ketiga target
# 'raw_values' -> menampilkan skor untuk masing-masing target
r2_avg = r2_score(y_test, y_pred, multioutput='uniform_average')
mae_avg = mean_absolute_error(y_test, y_pred, multioutput='uniform_average')
rmse_avg = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='uniform_average'))

r2_raw = r2_score(y_test, y_pred, multioutput='raw_values')
mae_raw = mean_absolute_error(y_test, y_pred, multioutput='raw_values')
rmse_raw = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))


print(f"R-squared (R²) Rata-rata           : {r2_avg:.4f}")
print(f"Mean Absolute Error (MAE) Rata-rata  : {mae_avg:.4f}")
print(f"Root Mean Squared Error (RMSE) Rata-rata: {rmse_avg:.4f}")
print("-" * 50)
print("SKOR UNTUK MASING-MASING TARGET:")
for i, target_name in enumerate(kolom_target):
    print(f"  -> Target: {target_name}")
    print(f"     R²  : {r2_raw[i]:.4f}")
    print(f"     MAE : {mae_raw[i]:.4f}")
    print(f"     RMSE: {rmse_raw[i]:.4f}")


Fitur yang digunakan: ['VESSEL ID (DMY)', 'BERTH LOCATION', 'Voyage No.', 'Voyage Yr', 'TOTAL BONGKARAN_FULL_40 HC', 'PENGAJUAN KE PLANNER_FULL_40 HC', 'ACC PENGAJUAN_FULL_40 HC']
Target yang akan diprediksi: ['REALISASI ALL DEPO_ALL DEPO_FXD_40 HC', 'SHIPSIDE_YES_FXD_40 HC', 'SHIPSIDE_NO_FXD_40 HC']

Membersihkan semua kolom numerik...
Baris dengan nilai target kosong telah dihapus.

Pipeline untuk preprocessing dan model Random Forest berhasil dibuat.

Data dibagi menjadi:
  - 1076 baris untuk training
  - 270 baris untuk testing

Memulai pencarian parameter terbaik untuk Multi-output...
Fitting 3 folds for each of 24 candidates, totalling 72 fits

HASIL EVALUASI MODEL MULTI-OUTPUT
Waktu training          : 28.68 detik
Parameter terbaik        : {'model__max_depth': 10, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 100}
R-squared (R²) Rata-rata           : 0.9726
Mean Absolute Error (MAE) Rata-rata  : 0.1744
Root Mean Squared Error (RMSE) Rata-r

In [47]:
import joblib

joblib.dump(best_pipeline, 'acc_realisasi_shipside_full_40.pkl')

['acc_realisasi_shipside_full_40.pkl']